# Manifold GP Semi-Supervised Learning via Precision Matrix on 2D Manifold

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
# from xvfbwrapper import Xvfb
# vdisplay = Xvfb(width=1920, height=1080)
# vdisplay.start()

In [2]:
import numpy as np
import torch
import gpytorch
from mayavi import mlab
from importlib.resources import files
from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from manifold_gp.utils.generate_truth import groundtruth_from_mesh

Select the device: GPU or CPU.

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Load Dataset and Generate Ground Truth

In [4]:
data_path = files('manifold_gp.data').joinpath('dragon10k.stl')
nodes, faces, truth = groundtruth_from_mesh(data_path)
sampled_x = torch.from_numpy(nodes).float().to(device)
sampled_y = torch.from_numpy(truth).float().to(device)
(m, n) = sampled_x.shape

## Normalization

In [5]:
# mu, std = sampled_x.mean(0), sampled_x.std(0)
# sampled_x.sub_(mu).div_(std)

## Apply Noise to the Dataset

In order to simulate real case scenario we can apply a normally distributed noise to our dataset. Here we can consider to apply the noise either to the sampled data points from the manifold or to the ground truth relative to each labeled points.

### Apply noise to the manifold
Setting the variable `manifold_noise` you can apply Gaussian noise to the sampled points from the manifold.

In [6]:
manifold_noise = 0.0
noisy_x = sampled_x + manifold_noise * torch.randn(m, n).to(device)

### Apply noise to the ground truth
Setting the variable `function_noise` you can apply Gaussian noise to the training points.

In [7]:
function_noise = 0.0
noisy_y = sampled_y + function_noise * torch.randn(m).to(device)

## Training Dataset

Create the training dataset. You can set the variable `num_train` to define the number of points within your dataset.

In [8]:
num_train = 100
train_idx = torch.randperm(m)[:num_train]
train_x = noisy_x[train_idx, :]
train_y = noisy_y[train_idx]

## Test Dataset

Create the training dataset. You can set the variable `num_test` to define the number of points within your dataset.

In [9]:
num_test = 100
test_idx = torch.randperm(m)[:num_test]
test_x = noisy_x[test_idx, :]
test_y = noisy_y[test_idx]

## Initialize Kernel

Create the Riemann Matern kernel. `nu` control the kernel smoothness like in the stadard Matern kernel. `neighbors` set the number of nearest neighbors in the KNN-based discrete approximation of the Laplace-Beltrami operator and construction of differentiable eigenfunctions. `modes` set the number of the eigenfunctions used to approximate the Riemann Matern kernel. As for classical *GPyTorch* kernels the Riemann Matern kernel can be decorated using with the *ScaleKernel* to have the signal variance parameter.

In [10]:
nu = 2
neighbors = 10
modes = 50
support_kernel = gpytorch.kernels.RBFKernel()
kernel = gpytorch.kernels.ScaleKernel(RiemannMaternKernel(nu=nu, nodes=train_x, neighbors=neighbors, modes=modes, support_kernel=support_kernel))

## Initialize Likelihood and GP Model

Define the type of likelihood and the GP model. The GP model takes as input the training set and and labels, the likelihood and the Riemann Matern Kernel.

In [11]:
likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.GreaterThan(1e-8))
model = RiemannGP(train_x, train_y, likelihood, kernel).to(device)

## Model Hyperparameters Setting

In [12]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-5,
    'covar_module.base_kernel.epsilon': 0.5,
    'covar_module.base_kernel.lengthscale': 0.5,
    'covar_module.outputscale': 1.0,
    'covar_module.base_kernel.support_kernel.lengthscale': 0.1
}
model.initialize(**hypers)

## Train Model

Train the GP model using the fast Lanczos precision matrix optimization. `lr` set the learning rate; `iters` defines the number of iteratios; `verbose` can be set to `True` for printing the parameters each iteration.

In [13]:
lr = 1e-2
iters = 100
verbose = True
loss = model.manifold_informed_train(lr, iters, verbose)

Iteration: 0, Loss: 129982.898, Noise Variance: 0.003, Signal Variance: 1.000, Lengthscale: 0.500, Epsilon: 0.500
Iteration: 1, Loss: 125269.453, Noise Variance: 0.003, Signal Variance: 1.003, Lengthscale: 0.504, Epsilon: 0.496
Iteration: 2, Loss: 120736.945, Noise Variance: 0.003, Signal Variance: 1.006, Lengthscale: 0.508, Epsilon: 0.492
Iteration: 3, Loss: 116381.453, Noise Variance: 0.003, Signal Variance: 1.009, Lengthscale: 0.512, Epsilon: 0.488
Iteration: 4, Loss: 112198.773, Noise Variance: 0.003, Signal Variance: 1.013, Lengthscale: 0.516, Epsilon: 0.484
Iteration: 5, Loss: 108184.758, Noise Variance: 0.003, Signal Variance: 1.016, Lengthscale: 0.520, Epsilon: 0.481
Iteration: 6, Loss: 104334.789, Noise Variance: 0.003, Signal Variance: 1.019, Lengthscale: 0.524, Epsilon: 0.477
Iteration: 7, Loss: 100644.258, Noise Variance: 0.003, Signal Variance: 1.022, Lengthscale: 0.528, Epsilon: 0.473
Iteration: 8, Loss: 97108.312, Noise Variance: 0.003, Signal Variance: 1.025, Lengthscal

Iteration: 76, Loss: 20520.211, Noise Variance: 0.004, Signal Variance: 1.164, Lengthscale: 0.737, Epsilon: 0.287
Iteration: 77, Loss: 20247.336, Noise Variance: 0.004, Signal Variance: 1.165, Lengthscale: 0.739, Epsilon: 0.285
Iteration: 78, Loss: 19980.967, Noise Variance: 0.004, Signal Variance: 1.166, Lengthscale: 0.741, Epsilon: 0.283
Iteration: 79, Loss: 19720.865, Noise Variance: 0.004, Signal Variance: 1.167, Lengthscale: 0.743, Epsilon: 0.281
Iteration: 80, Loss: 19466.812, Noise Variance: 0.004, Signal Variance: 1.169, Lengthscale: 0.745, Epsilon: 0.279
Iteration: 81, Loss: 19218.600, Noise Variance: 0.004, Signal Variance: 1.170, Lengthscale: 0.747, Epsilon: 0.277
Iteration: 82, Loss: 18976.020, Noise Variance: 0.004, Signal Variance: 1.171, Lengthscale: 0.749, Epsilon: 0.275
Iteration: 83, Loss: 18738.891, Noise Variance: 0.004, Signal Variance: 1.172, Lengthscale: 0.751, Epsilon: 0.272
Iteration: 84, Loss: 18507.021, Noise Variance: 0.004, Signal Variance: 1.173, Lengthsca

## Model Evaluation

In [14]:
%%capture
likelihood.eval()
model.eval()

Getting faster predictive distributions using [LOVE](https://arxiv.org/abs/1803.06058). We compute the **mean**, the **standard deviation** and one **posterior sample**. In addition we evaluate the kernel at the first point of our dataset.

In [15]:
%%capture
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    # GP posteriors
    preds = likelihood(model(noisy_x))
    mean = preds.mean.cpu().numpy()
    std = preds.stddev.cpu().numpy()
    posterior_sample = preds.sample().cpu().numpy()

    # Prior Variance
    var = kernel.base_kernel.variance(noisy_x).cpu().numpy()

    # Kernel evaluation
    point = 3405
    kernel_eval = kernel(noisy_x[point, :].unsqueeze(0), noisy_x).evaluate().squeeze().cpu().numpy()

    # Eigenfunctions
    mode = 1
    eigfunctions = kernel.base_kernel.eigenfunctions(noisy_x).cpu().numpy()

    # Bring data to numpy
    sampled_x = sampled_x.cpu().numpy()
    sampled_y = sampled_y.cpu().numpy()
    train_x = train_x.cpu().numpy()

## Plot Results

In [16]:
%%capture
%%bash
jupyter nbextension install --py mayavi --user

In [17]:
%%capture
mlab.init_notebook()
v_options = {'mode': 'sphere', 'scale_factor': 3e-3, 'color': (0, 0, 0)}

### Ground Truth with Sample Training Points

In [18]:
mlab.figure('Groud Truth', size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))
mlab.view(0.0, 180.0, 0.5139171204775793)
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=sampled_y)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:, 0], train_x[:, 1], train_x[:, 2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x05P\x08\x02\x00\x00\x00%a\xd0\x97\x…

### Mean

In [19]:
mlab.figure('Mean', size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))
mlab.view(0.0, 180.0, 0.5139171204775793)
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=mean)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:, 0], train_x[:, 1], train_x[:, 2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x05P\x08\x02\x00\x00\x00%a\xd0\x97\x…

### Standard Deviation

In [20]:
mlab.figure('Standard Deviation', size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))
mlab.view(0.0, 180.0, 0.5139171204775793)
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=std)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:, 0], train_x[:, 1], train_x[:, 2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x05P\x08\x02\x00\x00\x00%a\xd0\x97\x…

### One Posterior Sample

In [21]:
mlab.figure('One Posterior Sample', size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))
mlab.view(0.0, 180.0, 0.5139171204775793)
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=posterior_sample)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:, 0], train_x[:, 1], train_x[:, 2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x05P\x08\x02\x00\x00\x00%a\xd0\x97\x…

### Prior Variance

In [22]:
mlab.figure('Prior Variance', size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))
mlab.view(0.0, 180.0, 0.5139171204775793)
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=var)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:, 0], train_x[:, 1], train_x[:, 2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x05P\x08\x02\x00\x00\x00%a\xd0\x97\x…

### Kernel Evaluation

In [23]:
kernel_eval = kernel_eval - np.min(kernel_eval)
kernel_eval /= np.max(kernel_eval)
mlab.figure('Kernel Evaluation', size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))
mlab.view(0.0, 180.0, 0.5139171204775793)
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=kernel_eval)
mlab.colorbar(orientation='vertical')
mlab.points3d(sampled_x[point, 0], sampled_x[point, 1], sampled_x[point, 2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x05P\x08\x02\x00\x00\x00%a\xd0\x97\x…

### Eigenfunction

In [24]:
eigfun = eigfunctions[mode, :] - np.min(eigfunctions[mode, :])
eigfun /= np.max(eigfun)
mlab.figure('Eigenfunction', size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))
mlab.view(0.0, 180.0, 0.5139171204775793)
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=eigfun)
mlab.colorbar(orientation='vertical')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x05P\x08\x02\x00\x00\x00%a\xd0\x97\x…